In [1]:
import requests
import json
import pandas as pd
import time
#import convex as cx
import tmqa3 as tmqa

Autosaving every 180 seconds





Using TensorFlow backend.


Loading the params file
Input encoding {'o': 2, '{': 3, '.': 4, 'J': 5, '0': 6, '1': 7, '<': 8, 'B': 9, 'd': 10, '£': 11, 'e': 12, '6': 13, '!': 14, 'O': 15, 'M': 16, 'X': 17, 'f': 18, 't': 19, 'C': 20, 'V': 21, 'z': 22, 'K': 23, '\\': 24, '9': 25, 'P': 26, 'S': 27, '/': 28, '₹': 29, 'F': 30, 'G': 31, '=': 32, '8': 33, ')': 34, '+': 35, ']': 36, 'U': 37, "'": 38, '"': 39, 'g': 40, 'N': 41, 'r': 42, 'u': 43, '&': 44, '$': 45, 'x': 46, '%': 47, ':': 48, '@': 49, '^': 50, 'I': 51, 'L': 52, 'Z': 53, 'h': 54, 'W': 55, 'A': 56, 'v': 57, '?': 58, '2': 59, '~': 60, 's': 61, 'T': 62, 'R': 63, ',': 64, '|': 65, '4': 66, '>': 67, 'y': 68, '(': 69, '[': 70, 'k': 71, 'H': 72, 'l': 73, 'j': 74, '7': 75, 'n': 76, 'i': 77, 'D': 78, 'Q': 79, ' ': 80, 'm': 81, 'Y': 82, '*': 83, '}': 84, '#': 85, 'p': 86, 'q': 87, '5': 88, 'c': 89, '`': 90, 'a': 91, 'b': 92, 'w': 93, '3': 94, 'E': 95, ';': 96, '-': 97}
Input decoding {2: 'o', 3: '{', 4: '.', 5: 'J', 6: '0', 7: '1', 8: '<', 9: 'B', 10: 'd', 11: '£', 12: '

In [2]:
#f_test = open("/data/users/romain.claret/tm/wikidata-simplequestions/annotated_wd_data_test.txt",'r')
#out_test = f_test.readlines()

In [3]:
def pickle_data(df, filename):
    pd.set_option('display.max_columns', None)
    filename = "/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/"+filename+'.pickle.bz2'
    #df.summary = df.summary.map(sanitize_str)
    print("Done!",filename)
    return df.to_pickle(filename, compression='bz2')

In [4]:
#lang: en, fr, de, it, es, zh
#kb: dbpedia, wikidata, dblp, freebase
def ask_qanswer(question):
    data = {'query': question,'lang': 'en','kb': 'wikidata'}
    query = requests.post('http://qanswer-core1.univ-st-etienne.fr/api/gerbil', data=data)
    if not query:
        return False
    if (query.json()['questions'][0]['question']['answers']) == None:
        return False
    #if (query.json()['questions'][0]['question']['answers'].replace('\n', '')) == None:
    #    return False
    #print(query.json()['questions'][0]['question']['answers'].replace('\n', '').get("results"))
    try:
        response = (json.loads(query.json()
                .get("questions")[0]
                .get("question")
                .get("answers")
                .replace('\n', ''))
         .get("results").get("bindings"))
    except:
        return False
    
    if response:
        return response[0].get("o1").get("value")[len("http://www.wikidata.org/entity/"):] if response[0].get("o1") is not None else False
    else:
        return False

#ask_qanswer("Who is the wife of Barack Obama")
#ask_qanwser("Which equestrian was born in dublin?")
#ask_qanswer("what is the main language spoken in a ghentar si muore facile")
#ask_qanswer("was the film helpmates in color or black-and-white?")

In [5]:
def ask_platypus(question):
    headers = {'Accept': 'application/json','Accept-Language': 'en',}
    params = (('q', question),('lang', 'en'))

    response = requests.get('https://qa.askplatyp.us/v0/ask', headers=headers, params=params)
    if response:
        if type(response.json()['member']) is list:
            #print(response.json()['member'][0]['result'])
            if response.json()['member'] != []:
                if '@id' in (json.dumps(response.json()['member'][0]['result'])):
                    ps_result = (json.dumps(response.json()['member'][0]['result']['@id']))
                else: return False
            else: return False
        else:
            try:
                if '@id' in (json.dumps(response.json()['member']['result'])):
                    ps_result = (json.dumps(response.json()["member"]['result']['@id']))
                else: return False
            except:
                return False
    else: return False
    ps_result = ps_result[4:-1]
    #print(result[:1])
    if ps_result[:1] != 'P' and ps_result[:1] != 'Q':
        return False
    return ps_result
#ask_platypus("Which genre of album is harder.....faster?")
#ask_platypus("how does engelbert zaschka identify")
#ask_platypus("Which Swiss conductor's cause of death is myoc...")
#ask_platypus("where was padraic mcguinness's place of death")
#ask_platypus("was the film helpmates in color or black-and-white?")
#ask_platypus("Who created the show life on earth")
#ask_platypus("Who is the wife of Barack Obama")

In [6]:
def ask_convex(question):
    cx_result = cx.answer_complete_question(question, cx.tagmeToken)['answers'][0]['answer']
    #print(cx_result)
    #answer = str(cx.wd.wikidata_id_to_label(result['answers'][0]['answer']))
    try:
        if not cx_result:
            return cx_result
        if cx_result[:1] != 'P' and cx_result[:1] != 'Q':
            return False
        return cx_result
    except:
        return False

#ask_convex("Which actor voiced the Unicorn in The Last Unicorn?")
#ask_convex("Which genre of album is harder.....faster?")
#ask_convex("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#ask_convex("Who is the wife of Barack Obama")
#ask_convex("100% senorita is a television show in what language?")

In [7]:
def ask_tmqa(question, verbose=False, timer=False, show_graph=False, cores=tmqa.mp.cpu_count(), banning_str=False):
    tmqa_result = tmqa.answer_question(question, verbose=verbose, timer=timer, show_graph=show_graph, cores=cores, banning_str=banning_str)
    #print(tmqa_result)
    if not tmqa_result:
        return tmqa_result
    #if tmqa_result[]
    #if tmqa_result[0][0][:1] != 'P' and tmqa_result[0][0][:1] != 'Q':
    #    return False
    return tmqa_result[0]

#answer = ask_tmqa_1("Which actor voiced the Unicorn in The Last Unicorn?", verbose=True)
#answer = ask_tmqa_1("what's akbar tandjung's ethnicity", verbose=True)
#ask_tmqa_1("Which genre of album is harder.....faster?")
#ask_tmqa_1("Which label is somevelvetsidewalk signed to ttle of fort fisher ")
#answer = ask_tmqa("Who is the wife of Barack Obama")
#print(answer)

In [8]:
# Evaluate qanswer
#arguments[0] Subject
#arguments[1] Predicate
#arguments[2] Object
#arguments[3] question

#HEADERS = ['question', 'source', 'qanswer', 'platypus', 'convex', 'tmqa']
#df = pd.DataFrame(columns=HEADERS)

#start_time = time.time()
#out_test_len = len(out_test)
##print(out_test_len)
#for i, line in enumerate(out_test):
#    arguments = line.split("\t")
#    print("\r\t>>> Processing {}/{}: {}".format(i,out_test_len,arguments[3][:-1]), end='                                  ')
#    if i < len(df):
#        continue
#    result_qanswer = ask_qanswer(arguments[3][:-1])
#    result_platypus = ask_platypus(arguments[3][:-1])
#    result_convex = ask_convex(arguments[3][:-1])
#    #result_tmqa = ask_tmqa(arguments[3][:-1])[1]
#    print("\n",i, "-> qAnswer", result_qanswer) if result_qanswer == arguments[2] else False
#    print("\n",i, "-> Platypus", result_platypus) if result_platypus == arguments[2] else False
#    print("\n",i, "-> Convex", result_convex) if result_convex == arguments[2] else False
#    #print("\n",i, "-> TMqa", result_tmqa) if result_tmqa == arguments[2] else False
#    df = df.append({"question":arguments[3][:-1], 'source':arguments[2], 'qanswer':result_qanswer, 'platypus':result_platypus, 'convex':result_convex, 'tmqa1':result_tmqa}, ignore_index=True)
#
#end_time = time.time()
#print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
#print(df)

In [9]:
#len(df)

In [10]:
#df.tail(5)

In [11]:
#SAVING
#pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(len(df_loaded)))

In [12]:
#LOADING
#df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-9961.pickle.bz2")

#/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-5.pickle.bz2


In [13]:
#df = df_loaded

In [14]:
#df_loaded

In [15]:
#df_loaded = df_loaded.replace("", False)

In [16]:
#df_loaded['qanswer'][34] = False

In [17]:
#del df_loaded['tm2']

In [18]:
#df_loaded.rename({'mine':'tm1'}, axis=1, inplace=True)

In [19]:
#df_loaded['tm1_top4'] = "False"

In [20]:
#df_loaded = df_loaded[['question','source','qanswer','platypus','convex','tm1','tm1_time','tm1_top2','tm1_top3','tm1_top4','tm1_top5','tm1_topall']]

In [21]:
#df_loaded

In [22]:
#LOADING QAGraph

df_loaded = pd.read_pickle("/data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-534.pickle.bz2")



In [23]:
print(df_loaded.dtypes)

question      object
source        object
qanswer       object
platypus      object
convex        object
tm1           object
tm1_time      object
tm1_top2      object
tm1_top3      object
tm1_top4      object
tm1_top5      object
tm1_topall    object
dtype: object


In [ ]:
banning_str = False#[["ř","r"],["ø","o"]]

df_loaded_len = len(df_loaded)
for i, question in enumerate(df_loaded['question']):
    if i >= 535:
        source = str(df_loaded['source'][i])
        print(str(i)+"/"+str(df_loaded_len),question,"-> source:",source)
        
        start_time = time.time()
        result_tmqa = ask_tmqa(question, verbose=True, timer=True, banning_str=banning_str)
        
        if result_tmqa:
            df_loaded['tm1'][i] = result_tmqa[0]
            if source in result_tmqa[:2]:
                df_loaded['tm1_top2'][i] = True
            if source in result_tmqa[:3]:
                df_loaded['tm1_top3'][i] = True
            if source in result_tmqa[:4]:
                df_loaded['tm1_top4'][i] = True
            if source in result_tmqa[:5]:
                df_loaded['tm1_top5'][i] = True
            if source in result_tmqa:
                df_loaded['tm1_topall'][i] = True
        else:
            df_loaded['tm1'][i] = False
        end_time = time.time()
        df_loaded['tm1_time'][i] = round(end_time-start_time,2)
        print("->\tRunning time is {}s".format(round(end_time-start_time,2)))
        print(str(str(df_loaded['tm1'][i])==str(source)),"---> result_tmqa:",str(source))
        
        pickle_data(df_loaded, "benchmarking-qanswer-platypus-convex-graphqa-from-0-to-"+str(i))
        
        print(df_loaded.loc[i,:])
        print("\n")

535/9961 which type of people does roberto benigni belong to -> source: Q50001
User input: which type of people does roberto benigni belong to
--> Auto correcting question in progress...

-> Auto corrected q_nlp: Which type of people does Roberto Benigni belong to 
-> q_themes: ([(Roberto Benigni, ['Q23301']), (people, ['Q2472587', 'Q33659']), (Type, ['Q3707858', 'Q7860632']), (People, ['Q769695', 'Q11887416']), (type, ['Q21146257', 'Q1325930'])], [Which type of people does Roberto])
-> q_themes_enhanced: []
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: type
behold: get_most_similar started with: do
behold: get_most_similar started with: belong
-> q_predicates: [(does, []), (belong, []), (type, ['P427']), (people, ['P172', 'P1315'])]
-> q_predicates 	Running time is 16.46s
-> q_focused_parts: [(type, ['Q21146257', 'Q1325930']), (people, ['Q2472587', 'Q33659'])]
-> Building the graph with k_deep 3 ... (could 

-> Auto corrected q_nlp: Who is signed with MySpace records 
-> q_themes: ([(MySpace, ['Q40629', 'Q18612105']), (MySpace records, ['Q2040546'])], [be sign])
-> q_themes_enhanced: [('sign', ['Q1193832']), ('Sign', ['Q11246169']), ('Myspace', ['P3265'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: record
-> q_predicates: [(be, ['P31']), (signed, ['P1412']), (records, [])]
-> q_predicates 	Running time is 34.81s
-> q_focused_parts: [(MySpace records, ['Q2040546'])]
-> Building the graph with k_deep 3 ... (could be long)
->New graph 	Running time is 16.82s
-->  88 nodes and 88 edges
--> Removing meaningless subgraphs
--> New graph of: 88 nodes and 88 edges
---> Rebuilding the graph with k_deep 4 ... Previously: 88 nodes or 88 edges was below the limit of 100
->New graph 	Running time is 17.09s
-->  90 nodes and 90 edges
--> Removing meaningless subgraphs
--> New graph of: 90 nodes and 90 edges
---> Rebuilding 

->New graph 	Running time is 15.13s
-->  55 nodes and 54 edges
--> Removing meaningless subgraphs
--> New graph of: 55 nodes and 54 edges
---> Rebuilding the graph with k_deep 9 ... Previously: 55 nodes or 54 edges was below the limit of 100
->New graph 	Running time is 14.58s
-->  61 nodes and 60 edges
--> Removing meaningless subgraphs
--> New graph of: 61 nodes and 60 edges
---> Rebuilding the graph with k_deep 11 ... Previously: 61 nodes or 60 edges was below the limit of 100
->New graph 	Running time is 14.21s
-->  73 nodes and 72 edges
--> Removing meaningless subgraphs
--> New graph of: 73 nodes and 72 edges
---> Rebuilding the graph with k_deep 13 ... Previously: 73 nodes or 72 edges was below the limit of 100
->New graph 	Running time is 14.1s
-->  83 nodes and 82 edges
--> Removing meaningless subgraphs
--> New graph of: 83 nodes and 82 edges
---> Rebuilding the graph with k_deep 14 ... Previously: 83 nodes or 82 edges was below the limit of 100
->New graph 	Running time is 1

--> len(path_nodes): 40
->Computing possible paths 	Running time is 20.24s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 28
->	Running time is 3.14s
-> Computing hypothesises...
--> hypothesises: [['Q4104520', 34.011905819715935], ['Q4413495', 33.94951304395374], ['Q4150965', 33.84162601673425], ['Q201918', 2.7377328903656957], ['Q11366', 2.7377328903656957]]
->Computing hypothesises 	Running time is 7.46s
-> Computing golden paths...
--> len(golden_paths): 4
->	Running time is 3.45s
--> len(cleared_golden_paths): 2
---> First path: ['Q4104520', 'P136', 'Q483394', 'P642', 'Q17537576']
->	Total Running time is 175.8s

->	Running time is 176.01s
False ---> result_tmqa: Q837837
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-541.pickle.bz2
question             which genre does every avenue work under
source                                                Q837837
qanswer             

-> q_themes_enhanced: [('The Film', ['Q16679544']), ('The Love', ['Q11250791']), ('Film', ['Q25302710']), ('Love', ['Q1014414'])]
--> Calculating predicates... (could be long.. depends on uncached unpure predicates)
behold: get_most_similar started with: love
behold: get_most_similar started with: hurt
-> q_predicates: [(be, ['P31']), (hurts, []), (language, ['P1412', 'P364']), (film, ['P57']), (love, [])]
-> q_predicates 	Running time is 73.68s
-> q_focused_parts: [(language, ['Q315', 'Q34770'])]
-> Building the graph with k_deep 3 ... (could be long)
->New graph 	Running time is 29.93s
-->  178 nodes and 172 edges
--> Removing meaningless subgraphs
--> New graph of: 178 nodes and 172 edges
-> predicates_dict: {'P527': 1, 'P31': 30, 'P1013': 3, 'P155': 2, 'P156': 2, 'P175': 1, 'P2354': 1, 'P364': 1, 'P805': 1, 'P1343': 1, 'P735': 1, 'P509': 2, 'P1424': 1, 'P813': 1, 'P973': 1, 'P1709': 1, 'P1245': 1, 'P361': 4, 'P282': 1, 'P1535': 1, 'P279': 1, 'P577': 1, 'P2633': 1, 'P264': 3, 'P3744

--> len(path_nodes): 1542
->Computing possible paths 	Running time is 24.33s
-> Filtering paths... (could be long)
--> len(paths_nodes_filtered): 1164
->	Running time is 3.08s
-> Computing hypothesises...
--> hypothesises: [['Q5287', 4.936739324284452], ['Q158052', 1.1266725452098396], ['Q11424', 0.9479735287534822], ['Q1860', 0.8643445971230035], ['1988-11-21T00:00:00Z', 0.7477694094463193], ['2005-01-01T00:00:00Z', 0.419676383891622], ['Q101583', -0.18187212039333625]]
->Computing hypothesises 	Running time is 29.72s
-> Computing golden paths...
--> len(golden_paths): 7
->	Running time is 14.62s
--> len(cleared_golden_paths): 4
---> First path: ['Q5287', 'P407', 'Q466024', 'P585', '2005-01-01T00:00:00Z']
->	Total Running time is 177.74s

->	Running time is 177.96s
False ---> result_tmqa: Q460946
Done! /data/users/romain.claret/tm/mse.tm.chatbot.base/benchmark_pickles/wd-sq/benchmarking-qanswer-platypus-convex-graphqa-from-0-to-546.pickle.bz2
question                         who was t

In [ ]:
df_loaded.head(10)

In [ ]:
df = df_loaded.copy()
#df = df.replace("", False)

In [ ]:
max_row = 150
df_len = len(df)
df_qanswer_max = df[(df.index<=max_row) & (df.qanswer == df.source)]
df_qanswer_max_len = len(df_qanswer_max)

df_platypus_max = df[(df.index<=max_row) & (df.platypus == df.source)]
df_platypus_max_len = len(df_platypus_max)

df_convex_max = df[(df.index<=max_row) & (df.convex == df.source)]
df_convex_max_len = len(df_convex_max)

df_tm1_max = df[(df.index<=max_row) & (df.tm1 == df.source)]
df_tm1_max_len = len(df_tm1_max)

df_tm1_max_top2 = df[(df.index<=max_row) & (df.tm1_top2 == True)]
df_tm1_max_top2_len = len(df_tm1_max_top2)

df_tm1_max_top3 = df[(df.index<=max_row) & (df.tm1_top3 == True)]
df_tm1_max_top3_len = len(df_tm1_max_top3)

df_tm1_max_top4 = df[(df.index<=max_row) & (df.tm1_top4 == True)]
df_tm1_max_top4_len = len(df_tm1_max_top4)

df_tm1_max_top5 = df[(df.index<=max_row) & (df.tm1_top5 == True)]
df_tm1_max_top5_len = len(df_tm1_max_top5)

df_tm1_max_topall = df[(df.index<=max_row) & (df.tm1_topall == True)]
df_tm1_max_topall_len = len(df_tm1_max_topall)

print("qanswer:", df_qanswer_max_len,df_qanswer_max_len/max_row)
print("platypus:", df_platypus_max_len, df_platypus_max_len/max_row)
print("convex:", df_convex_max_len, df_convex_max_len/max_row)
print("tm1:", df_tm1_max_len, df_tm1_max_len/max_row)
print("tm1_top2:", df_tm1_max_top2_len, df_tm1_max_top2_len/max_row)
print("tm1_top3:", df_tm1_max_top3_len, df_tm1_max_top3_len/max_row)
print("tm1_top4:", df_tm1_max_top4_len, df_tm1_max_top4_len/max_row)
print("tm1_top5:", df_tm1_max_top5_len, df_tm1_max_top5_len/max_row)
print("tm1_topall:", df_tm1_max_topall_len, df_tm1_max_topall_len/max_row)

#df[ & (df.qanswer == df.source)]

In [ ]:
print("qanswer:",len(df[df.qanswer == df.source]),len(df[df.qanswer == df.source])/len(df))
print("platypus:",len(df[df.platypus == df.source]),len(df[df.platypus == df.source])/len(df))
print("convex:",len(df[df.convex == df.source]),len(df[df.convex == df.source])/len(df))
print("tm1:",len(df[df.tm1 == df.source]),len(df[df.tm1 == df.source])/len(df))
print("tm1_top2:",len(df[df.tm1_top2 == True]),len(df[df.tm1_top2 == True])/len(df))
print("tm1_top3:",len(df[df.tm1_top3 == True]),len(df[df.tm1_top3 == True])/len(df))
print("tm1_top4:",len(df[df.tm1_top4 == True]),len(df[df.tm1_top4 == True])/len(df))
print("tm1_top5:",len(df[df.tm1_top5 == True]),len(df[df.tm1_top5 == True])/len(df))
print("tm1_topall:",len(df[df.tm1_topall == True]),len(df[df.tm1_topall == True])/len(df))

In [ ]:
#df.tm1_top2 = df.tm1_top3